In [3]:
#import pymongo
import pandas as pd

C:\Users\rafal\AppData\Local\Temp\ipykernel_9452\2659123430.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Connect to the local MongoDB instance
# client = pymongo.MongoClient('localhost', 27017)

In [3]:
# # Access the 'dd-usda' database
# db = client['dd-usda']

# # Access the 'publications' collection
# collection = db['publications']

In [4]:
# # Define the query parameters
# query = {
#     'author_american': True,
#     '$or': [
#         {
#             'aliases': {
#                 '$in': [
#                     "USDA", "USDA Census", "US Department of Agriculture",
#                     "United States Department of Agriculture", "NASS", "NASS Census of Agriculture"
#                 ]
#             }
#         },
#         {
#             'second_alias': {
#                 '$in': [
#                     "USDA", "USDA Census", "US Department of Agriculture",
#                     "United States Department of Agriculture", "NASS", "NASS Census of Agriculture"
#                 ]
#             }
#         }
#     ]
# }

In [5]:
# # Execute the query
# projection = {
#     'name': 1,
#     'doi': 1,
#     'year': 1,
#     '_id': 0  # Exclude '_id' if you don't need it
# }
# cursor = collection.find(query, projection)

In [6]:
# Convert cursor to list
# documents = list(cursor)

In [5]:
openalex_pubs_pubs = pd.read_csv("openalex_pubs_pubs.csv")

In [6]:
scopus_pubs = pd.read_csv('scopus_pubs.csv')

In [7]:
scopus_pubs['doi'] = 'https://doi.org/' + scopus_pubs['doi'].astype(str)
print(scopus_pubs.head())

                                         doi  \
0   https://doi.org/10.1175/WCAS-D-17-0067.1   
1  https://doi.org/10.1061/9780784481400.026   
2  https://doi.org/10.1186/s13007-018-0338-z   
3         https://doi.org/10.1111/anti.12694   
4           https://doi.org/10.1111/csp2.489   

                                               title  year  
0  Tracking drought perspectives: A rural case st...  2018  
1   A Critical Analysis of the Lake Champlain's TMDL  2018  
2  Remote estimation of rapeseed yield with unman...  2018  
3  Pathogenic Metabolisms: A Rift and the Zika Vi...  2021  
4  Exploring pathways to participation in an at-r...  2021  


In [8]:
print(openalex_pubs_pubs.head())

   Unnamed: 0                                              title  \
0           0  Research topics to scale up cover crop use: Re...   
1           1  The Impact of Targeted Data Collection on Nonr...   
2           2  Climate change beliefs, risk perceptions, and ...   
3           3  National-scale soybean mapping and area estima...   
4           4  Measuring land-use and land-cover change using...   

                         doi  year  exists_in_openalex  
0      10.2489/jswc.72.3.59a  2017                True  
1      10.1515/jos-2017-0039  2017                True  
2  10.1016/j.crm.2016.11.004  2017                True  
3  10.1016/j.rse.2017.01.008  2017                True  
4  10.1016/j.jag.2017.06.007  2017                True  


In [9]:
import pandas as pd

# --- Standardize DOI Columns ---

# Remove 'https://doi.org/' prefix and standardize case in 'openalex'
openalex_pubs['doi'] = openalex_pubs['doi'].str.strip().str.lower()
openalex_pubs['doi'] = openalex_pubs['doi'].str.replace('https://doi.org/', '', regex=False)

# Remove 'https://doi.org/' prefix and standardize case in 'scopus'
scopus_pubs['doi'] = scopus_pubs['doi'].str.strip().str.lower()
scopus_pubs['doi'] = scopus_pubs['doi'].str.replace('https://doi.org/', '', regex=False)

# --- Add Indicator Columns ---

# 'openalex' indicator
openalex_pubs['exists_in_openalex'] = True

# 'scopus' indicator
scopus_pubs['exists_in_scopus'] = True

# --- Rename Columns for Consistency ---

# Rename 'name' to 'title' in 'openalex' if necessary
if 'name' in openalex_pubs.columns:
    openalex_pubs.rename(columns={'name': 'title'}, inplace=True)

# --- Merge DataFrames ---

# Perform an outer merge on 'doi'
merged_df = pd.merge(
    openalex_pubs,
    scopus_pubs,
    on='doi',
    how='outer',
    suffixes=('_openalex', '_scopus')
)

# --- Handle Missing Indicator Values ---

# Fill NaN values with False
merged_df['exists_in_openalex'] = merged_df['exists_in_openalex'].fillna(False)
merged_df['exists_in_scopus'] = merged_df['exists_in_scopus'].fillna(False)

# --- Consolidate 'title' and 'year' Columns ---

# Combine 'title' columns
merged_df['title'] = merged_df['title_openalex'].combine_first(merged_df['title_scopus'])

# Combine 'year' columns
merged_df['year'] = merged_df['year_openalex'].combine_first(merged_df['year_scopus'])

# --- Select Desired Columns ---

# Create the final DataFrame
final_df = merged_df[['doi', 'title', 'year', 'exists_in_openalex', 'exists_in_scopus']]

# --- Handle Data Types ---

# Convert 'year' to integer type if possible
final_df['year'] = final_df['year'].astype('Int64')

# --- Verify the Final DataFrame ---

print(final_df.head())


                                  doi  \
0      10.1001/jamadermatol.2022.0136   
1   10.1001/jamahealthforum.2023.1672   
2   10.1001/jamahealthforum.2023.2247   
3  10.1001/jamanetworkopen.2022.30697   
4  10.1001/jamanetworkopen.2022.36898   

                                               title  year  \
0  Training and Practice Characteristics of Inter...  2022   
1  Changes in Self-Reported Adult Health and Hous...  2023   
2  Urine Drug Screening in a Telehealth Setting f...  2023   
3  Association of Neighborhood Economic Trajector...  2022   
4  Association of National Expansion of Insurance...  2022   

   exists_in_openalex  exists_in_scopus  
0                True             False  
1                True             False  
2                True             False  
3                True             False  
4                True             False  


C:\Users\rafal\AppData\Local\Temp\ipykernel_9452\3426958663.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['exists_in_openalex'] = merged_df['exists_in_openalex'].fillna(False)
C:\Users\rafal\AppData\Local\Temp\ipykernel_9452\3426958663.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['exists_in_scopus'] = merged_df['exists_in_scopus'].fillna(False)
C:\Users\rafal\AppData\Local\Temp\ipykernel_9452\3426958663.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [11]:
# Step 1: Create 'source' column
def determine_source(row):
    if row['exists_in_openalex'] and row['exists_in_scopus']:
        return 'Both'
    elif row['exists_in_openalex']:
        return 'Only openalex'
    elif row['exists_in_scopus']:
        return 'Only Scopus'
    else:
        return 'Neither'  # Should not happen based on your data

final_df['source'] = final_df.apply(determine_source, axis=1)

# Step 2: Group by 'year' and 'source' and count
counts = final_df.groupby(['year', 'source']).size().reset_index(name='publication_count')

# Step 3: Sort and display the counts
counts = counts.sort_values(by=['year', 'source'])
print("Grouped Counts by Year and Source:")
print(counts)

# Step 4: Pivot the DataFrame for better visualization
pivot_table = counts.pivot(index='year', columns='source', values='publication_count').fillna(0).astype(int)

print("\nPivot Table of Counts:")
print(pivot_table)


Grouped Counts by Year and Source:
    year         source  publication_count
0   2017           Both                125
1   2017    Only Scopus                455
2   2017  Only openalex               1147
3   2018           Both                161
4   2018    Only Scopus                464
5   2018  Only openalex               1183
6   2019           Both                139
7   2019    Only Scopus                583
8   2019  Only openalex               1035
9   2020           Both                 64
10  2020    Only Scopus                644
11  2020  Only openalex                578
12  2021           Both                 76
13  2021    Only Scopus                749
14  2021  Only openalex                643
15  2022           Both                 54
16  2022    Only Scopus                670
17  2022  Only openalex                850
18  2023           Both                 64
19  2023    Only Scopus                466
20  2023  Only openalex                941
21  2024  Only open

C:\Users\rafal\AppData\Local\Temp\ipykernel_9452\276038858.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['source'] = final_df.apply(determine_source, axis=1)
